In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import RobustScaler
import os

# --- CẤU HÌNH ---
TEST_FILE_PATH = 'du_lieu_test_no_label.csv'  
OUTPUT_CSV = 'ket_qua_du_doan.csv'
AOI_EYES_X = 512; AOI_EYES_Y = 350
AOI_MOUTH_X = 512; AOI_MOUTH_Y = 550

print(f"⏳ Đang xử lý file dự đoán: {TEST_FILE_PATH}")

try:
    df_test = pd.read_csv(TEST_FILE_PATH)
    
    # 1. Tính AOI
    df_test['DIST_EYES'] = np.sqrt((df_test['FIX_X'] - AOI_EYES_X)**2 + (df_test['FIX_Y'] - AOI_EYES_Y)**2)
    df_test['DIST_MOUTH'] = np.sqrt((df_test['FIX_X'] - AOI_MOUTH_X)**2 + (df_test['FIX_Y'] - AOI_MOUTH_Y)**2)

    # 2. Pivot & Prepare
    def pivot_feature_test(df, value_col):
        # Lưu ý: File test thường không có cột 'label', nên chỉ group theo ID
        keys = ['numeric_id', 'image_id']
        if 'label' in df.columns: keys.append('label')
            
        df_pivoted = df.pivot_table(index=keys, columns='FIX_INDEX', values=value_col).reset_index()
        step_cols = [i for i in range(1, 21)]
        df_pivoted[step_cols] = df_pivoted[step_cols].bfill(axis=1).ffill(axis=1)
        return df_pivoted, df_pivoted[step_cols].values

    # Pivot các đặc trưng
    df_meta_test, pupil_test = pivot_feature_test(df_test, 'FIX_PUPIL')
    # Baseline correction cho Pupil
    pupil_test = pupil_test - pupil_test[:, 0].reshape(-1, 1)

    _, dur_test = pivot_feature_test(df_test, 'FIX_DURATION')
    _, deye_test = pivot_feature_test(df_test, 'DIST_EYES')
    _, dmou_test = pivot_feature_test(df_test, 'DIST_MOUTH')

    # Stack thành 3D (N, 20, 4)
    X_test_final = np.stack([pupil_test, dur_test, deye_test, dmou_test], axis=2)
    ids_test_final = df_meta_test['numeric_id'].values
    
    print(f"Dữ liệu test đã sẵn sàng: {X_test_final.shape}")

    # 3. Chuẩn hóa (Fit trên chính dữ liệu test)
    for i in range(4):
        scaler = RobustScaler()
        X_test_final[:, :, i] = scaler.fit_transform(X_test_final[:, :, i])

    # 4. Dự đoán (Ensemble 5 Models)
    total_probs = np.zeros(len(X_test_final))
    
    print(" Đang chạy dự đoán với 5 models...")
    for fold in range(1, 6):
        model_name = f'saved_models/model_fold_{fold}.keras'
        if os.path.exists(model_name):
            model = tf.keras.models.load_model(model_name)
            pred = model.predict(X_test_final, verbose=0).flatten()
            total_probs += pred
            print(f"   -> Model Fold {fold}: Xong.")
        else:
            print(f"   Không tìm thấy {model_name}")

    # Lấy trung bình cộng xác suất
    avg_probs = total_probs / 5.0

    # 5. Gom kết quả & Xuất file
    results_df = pd.DataFrame({
        'numeric_id': ids_test_final,
        'probability': avg_probs
    })

    # Tính trung bình cho từng người
    final_report = results_df.groupby('numeric_id').agg(
        avg_probability=('probability', 'mean')
    ).reset_index()

    # Ra quyết định ( > 0.5 là ASD)
    final_report['Prediction'] = (final_report['avg_probability'] > 0.5).astype(int)
    final_report['Label_Name'] = final_report['Prediction'].map({0: 'Bình thường', 1: 'ASD'})

    final_report.to_csv(OUTPUT_CSV, index=False)
    print(f"\nĐÃ XUẤT FILE KẾT QUẢ: {OUTPUT_CSV}")
    print(final_report.head())

except Exception as e:
    print(f" Lỗi: {e}")

⏳ Đang xử lý file dự đoán: du_lieu_test_no_label.csv
Dữ liệu test đã sẵn sàng: (4792, 20, 4)
 Đang chạy dự đoán với 5 models...
   -> Model Fold 1: Xong.
   -> Model Fold 2: Xong.
   -> Model Fold 3: Xong.
   -> Model Fold 4: Xong.
   -> Model Fold 5: Xong.

ĐÃ XUẤT FILE KẾT QUẢ: ket_qua_du_doan.csv
   numeric_id  avg_probability  Prediction   Label_Name
0           0         0.343985           0  Bình thường
1           1         0.843750           1          ASD
2           2         0.607590           1          ASD
3           3         0.318201           0  Bình thường
4           4         0.475473           0  Bình thường
